In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

project_path = '/content/drive/MyDrive/YT-Notes'

# Create the folder if it doesn't exist
if not os.path.exists(project_path):
    os.makedirs(project_path)
    print(f"Folder '{project_path}' created successfully!")
else:
    print(f"Folder '{project_path}' already exists.")


Folder '/content/drive/MyDrive/YT-Notes' already exists.


In [ ]:
!pip install youtube-transcript-api google-generativeai chromadb

Import modules

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api.formatters import TextFormatter

import google.generativeai as genai

import chromadb
from chromadb.utils import embedding_functions

Set up resources

In [14]:
from google.colab import userdata
import os

# Retrieve the Secrets
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
CHROMA_GOOGLE_GENAI_API_KEY = userdata.get('CHROMA_GOOGLE_GENAI_API_KEY')

# Set them as environment variables
os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY
os.environ['CHROMA_GOOGLE_GENAI_API_KEY'] = CHROMA_GOOGLE_GENAI_API_KEY

In [15]:
# Instantiate Gemini model
gemini_model = genai.GenerativeModel('models/gemini-1.5-flash')

# Load the vector database
chroma_client = chromadb.PersistentClient(path='/content/drive/MyDrive/YT-Notes/my_vectordb')

# Select an embedding function
gemini_ef = embedding_functions.GoogleGenerativeAiEmbeddingFunction(api_key=GEMINI_API_KEY)

# Load collection
chroma_collection = chroma_client.get_or_create_collection(name='yt_notes', embedding_function=gemini_ef)

Inputs:

In [25]:
# Some youtube videos:
# https://www.youtube.com/watch?v=RoR4XJw8wIc
# https://www.youtube.com/watch?v=3E0dqwQwX4k
# https://www.youtube.com/watch?v=eE6yvtKLwvk

yt_video_id = 'eE6yvtKLwvk'

# Add a prompt
prompt = "Extract the key points from the video transcript."

Extract transcript

In [26]:
transcript = YouTubeTranscriptApi.get_transcript(yt_video_id, languages=['en', 'en-US', 'en-GB'])
formatted_transcript = "\n".join(entry['text'] for entry in transcript)

with open("/content/drive/MyDrive/YT-Notes/temp_transcript.txt", "w", encoding='utf-8') as file:
    file.write(formatted_transcript)

Generate notes

In [27]:
response = gemini_model.generate_content(prompt + formatted_transcript, stream=False)

with open("/content/drive/MyDrive/YT-Notes/temp_notes.txt", "w", encoding='utf-8') as file:
    file.write(response.text)

Save notes

In [28]:
with open("/content/drive/MyDrive/YT-Notes/temp_notes.txt", "r", encoding='utf-8') as file:
    notes = file.read()

# Insert/update the notes in the vector database
chroma_collection.upsert(
    documents=[notes],
    ids=[yt_video_id]
)

# Validate the insertion
result = chroma_collection.get(yt_video_id, include=['documents'])
result

{'ids': ['eE6yvtKLwvk'],
 'embeddings': None,
 'documents': ['This video transcript provides a roadmap for learning AI and machine learning, categorized into key areas and recommending specific resources. Here\'s a summary of the key points:\n\n\n**I. Programming and Software Engineering:**\n\n* **Python:**  Essential for AI, used extensively in machine learning and AI infrastructure.  The speaker recommends the "Learn Python" course (freeCodeCamp), "Python for Everybody" specialization (Coursera), HackerRank, and LeetCode for practice and interview prep.  Neat Code is suggested for data structures, algorithms, and system design.  Harvard\'s CS50 is recommended for complete beginners.  **Key takeaway: Practice is paramount.**\n\n**II. Maths and Stats:**\n\n* **Essential Math Skills:**  While some argue it\'s not strictly necessary, the speaker emphasizes a strong foundation in statistics, linear algebra, and calculus for advanced AI work.  They recommend:\n    * *Practical Statistics f

Let's see the database records

In [36]:
# Fetch from 'yt_notes' collection
results = chroma_collection.get(include=["documents", "embeddings"])
print("IDs:", results["ids"])
print("Documents:", results["documents"])

IDs: ['RoR4XJw8wIc', '3E0dqwQwX4k', 'eE6yvtKLwvk']
Documents: ['LangChain is a popular Python framework for building AI applications with large language models (LLMs).  Key features and benefits include:\n\n* **Ease of building end-to-end LLM applications:**  It simplifies the process of combining LLMs with various data sources (databases, PDFs, emails), managing prompts, storing conversation history, and integrating multiple models or external tools.\n\n* **Composability:** It allows developers to combine different components (LLMs, prompts, memory, indices, agents) in a modular and flexible way.\n\n* **Six Key Modules:**\n    * **LLMs:** Provides a unified interface for accessing various LLMs (OpenAI, Hugging Face, Cohere, etc.).\n    * **Prompts:** Offers tools for prompt management, optimization, and serialization.\n    * **Chains:** Enables sequences of LLM calls, beyond single interactions.\n    * **Memory:** Provides a standard interface and implementations for storing conversat

In [35]:
import pandas as pd

df = pd.DataFrame({
    "ID": results["ids"],
    "Document": results["documents"],
    "Embedding Length": [len(vec) for vec in results["embeddings"]]
})

df.head()

,ID,Document,Embedding Length
0,RoR4XJw8wIc,LangChain is a popular Python framework for bu...,768
1,3E0dqwQwX4k,This video outlines a system for effective pro...,768
2,eE6yvtKLwvk,This video transcript provides a roadmap for l...,768


Search notes

In [40]:
query_text = "What are the key modules that LangChain provide for developers?"
n_results = 3
results = chroma_collection.query(
    query_texts=[query_text],
    n_results=n_results,
    include=['documents', 'distances', 'metadatas']
)

for i in range(len(results['ids'][0])):
  id = results['ids'][0][i]
  document = results['documents'][0][i]
  print("*********************************************************************")
  print(f"{i+1}. https://youtu.be/{id}")
  print("*********************************************************************")
  print(document)

*********************************************************************
1. https://youtu.be/RoR4XJw8wIc
*********************************************************************
LangChain is a popular Python framework for building AI applications with large language models (LLMs).  Key features and benefits include:

* **Ease of building end-to-end LLM applications:**  It simplifies the process of combining LLMs with various data sources (databases, PDFs, emails), managing prompts, storing conversation history, and integrating multiple models or external tools.

* **Composability:** It allows developers to combine different components (LLMs, prompts, memory, indices, agents) in a modular and flexible way.

* **Six Key Modules:**
    * **LLMs:** Provides a unified interface for accessing various LLMs (OpenAI, Hugging Face, Cohere, etc.).
    * **Prompts:** Offers tools for prompt management, optimization, and serialization.
    * **Chains:** Enables sequences of LLM calls, beyond single inter

In [41]:
prompt = "Answer the following question using document as context."
prompt += f"Question: {query_text}"
prompt += f"Document: {results['documents'][0][0]}"
response = gemini_model.generate_content(prompt, stream=False)
print(response.text)

LangChain provides six key modules: LLMs, Prompts, Chains, Memory, Indices, and Agents & Tools.

